# Данные

In [1]:
!wget https://www.dropbox.com/s/nd7v1fod89xla6j/vk_texts_with_sources.csv

--2019-05-29 21:13:21--  https://www.dropbox.com/s/nd7v1fod89xla6j/vk_texts_with_sources.csv
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.70.1, 2620:100:6026:1::a27d:4601
Подключение к www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 301 Moved Permanently
Адрес: /s/raw/nd7v1fod89xla6j/vk_texts_with_sources.csv [переход]
--2019-05-29 21:13:22--  https://www.dropbox.com/s/raw/nd7v1fod89xla6j/vk_texts_with_sources.csv
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uca976fd562c538cb46e54f99031.dl.dropboxusercontent.com/cd/0/inline/Ah21DsxdPCpDGcRvEmjX7WQhM6saL_c2Hbk2FS-eJktwRFaC3pO4Ugs0w8zsWnV8nzyqCKxG6zi0TGAsCKa83DseAvEmLF7cf0e1nBU_g-llhA/file# [переход]
--2019-05-29 21:13:23--  https://uca976fd562c538cb46e54f99031.dl.dropboxusercontent.com/cd/0/inline/Ah21DsxdPCpDGcRvEmjX7WQhM6saL_c2Hbk2FS-eJktwRFaC3pO4Ugs0w8zsWnV8nzyqCKxG6zi0TGAsCKa

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import pandas as pd
import numpy as np
df = pd.read_csv('vk_texts_with_sources.csv', usecols = ['text', 'source'])
df.head()

,text,source
0,Начальник Главного оперативного управления Ген...,mil
1,Артиллерийские подразделения общевойскового об...,mil
2,Подразделения морской пехоты Каспийской флотил...,mil
3,Команды на всеармейских этапах конкурсов АрМИ-...,mil
4,На большом учебно-методическом командирском сб...,mil


# Домашнее задание

В этом домашнем задании вы будете решать задачу тематической классификации. Даны тексты, опубликованные в нескольких пабликах VK.com, посвященных государственным и муниципальным службам. Формально задача заключается в том, чтобы по тексту ($d$) определить в каком паблике он опубликован, то есть, к какому классу $c$ он принадлежит. 

## Задание 1 [1 балл]. Описательные статистики
Посчитайте:
* количество текстов и количество классов
* количество слов (без лемматизации и с лемматизацией) в коллекции
* среднюю длину текста в словах и символах
* найдите 5 самых частых существительных в текстах каждого паблика 

*Рекомендуем использовать pandas для расчета описательных статистик.*

In [8]:
df.describe(include=[np.object])

,text,source
count,11526,11625
unique,11149,4
top,#СоветМЧС #МЧС #МЧСРОССИИ,mchsgov
freq,45,3030


У нас 4 класса и 11625 различных текстов.

In [36]:
import nltk
import re
from pymystem3 import Mystem

nltk.download('punkt')
mystem_analyzer = Mystem()

lemmatized = []
unlemmatized = []

for doc in df['text']:
    text = re.sub("<br>", "", str(doc))
    word_list = nltk.word_tokenize(text)
    unlemmatized.append(word_list)
    lemmatized.append([mystem_analyzer.lemmatize(word)[0] for word in word_list])

[nltk_data] Downloading package punkt to /home/valeria/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [37]:
print(unlemmatized[0][:10])
print("--------")
print(lemmatized[0][:10])

['Начальник', 'Главного', 'оперативного', 'управления', 'Генерального', 'штаба', 'Вооруженных', 'Сил', 'РФ', 'генерал-полковник']
--------
['начальник', 'главный', 'оперативный', 'управление', 'генеральный', 'штаб', 'вооруженный', 'сила', 'рф', 'генерал-полковник']


In [50]:
from itertools import chain

count_unlem = 0

for el in unlemmatized:
    count_unlem += len(el)

cont_lemmatized = list(chain.from_iterable(lemmatized))

print("Количество токенов в датасете:", count_unlem)
print("Количество уникальных лемм:", len(set(cont_lemmatized)))

Количество токенов в датасете: 1163263
Количество уникальных лемм: 33153


In [41]:
words_per_doc = np.mean([len(text) for text in unlemmatized])
symbols_per_doc = np.mean([len(str(doc)) for doc in df["text"]])

print("Среднее количество слов на документ:", int(words_per_doc))
print("Среднее количество символов на документ:", int(symbols_per_doc))

Среднее количество слов на документ: 100
Среднее количество символов на документ: 684


In [ ]:
from collections import Counter
from pymorphy2 import MorphAnalyzer
from tqdm import tqdm

morph = MorphAnalyzer()

tops = Counter()

for i in tqdm(range(len(df['source']))):
    for word in range(len(unlemmatized[i])):
        if morph.tag(unlemmatized[i][word])[0].POS == 'NOUN':
            tops.update([(unlemmatized[i][word], df["source"][i])])

Разделите коллекцию текстов на обучающую и тестовую части. 

In [104]:
# for i in range(len(tops)):
#     for el in tops[i].most_common(5):
#         print(el)
#     print("\n")

Counter({('МВД', 'mospolice'): 4947, ('России', 'mospolice'): 3890, ('МЧС', 'mchsgov'): 3486, ('полиции', 'mospolice'): 2977, ('Москве', 'mospolice'): 2674, ('полиция', 'mospolice'): 2616, ('России', 'russianpost'): 2535, ('Минобороны', 'mil'): 1813, ('УВД', 'mospolice'): 1669, ('России', 'mchsgov'): 1588, ('новости', 'mospolice'): 1511, ('МЧСРОССИИ', 'mchsgov'): 1226, ('России', 'mil'): 1179, ('Москвы', 'mospolice'): 1168, ('Почты', 'russianpost'): 1130, ('года', 'mospolice'): 1114, ('ПочтаРоссии', 'russianpost'): 936, ('службы', 'mospolice'): 896, ('Сотрудники', 'mospolice'): 855, ('Почта', 'russianpost'): 826, ('обороны', 'mil'): 799, ('дел', 'mospolice'): 791, ('военного', 'mil'): 736, ('мвдроссии', 'mospolice'): 727, ('сотрудники', 'mospolice'): 711, ('МВДРоссии', 'mospolice'): 705, ('ходе', 'mil'): 697, ('полицияроссии', 'mospolice'): 690, ('сотрудников', 'mospolice'): 687, ('РФ', 'mospolice'): 681, ('года', 'russianpost'): 678, ('начальника', 'mospolice'): 664, ('району', 'mospo

 ## Задание 2 [2 балла]. Классификация по правилам
 
 * Разработайте несколько правил вида "Если встречается слово $w$, то текст относится к паблику $c$"
 * Посчитайте, какую точность, полноту, $f$-меру и $accuracy$ вы получаете при классификации по правилам
 * Получилось ли у вас придумать правило, которое никогда не ошибается?

## Задание 3 [3 балла]. Baseline
Используйте стандартный ```sklearn.pipeline``` для классификации текстов: 
* векторизация 
* $tf-idf$ взвешивание 
* ваш любимый метод классификации.


Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

## Задание 4 [2 балла]. Снижение размерности
Добавьте в ваш ```sklearn.pipeline```  методы снижения размерности:  PCA / LSI / LSA / LDA / другое. Какие методы классификации разумно использовать после снижения размерности? Как изменились результаты классификации после добавления нового шага?

## Задание 5 [1 балл]. Лемматизация
Посмотрите, как влияет лемматизация на качество классификации. Как изменится качество классификации, если вы используете ```CountVectorizer``` на словах или $n$-граммах на лемматизированных текстах?

## Задание 6 [3 балла]. Continious bag of words
Для baseline решения мы использовали обычное представление текста в виде мешка слов. Попробуйте использовать другие модели представления текста – например, в виде непрерывного мешка слов, то есть, в виде набора эмбеддингов. Для того, чтобы получить вектор текста попробуйте:
* усреднить все эмбеддинги слов, входящих в этот текст
* усреднить все эмбеддинги слов, входящих в этот текст с $tf-idf$ весами
* использовать любую модель эмбеддинга документа.

Используйте любую модель эмбеддингов по вашему вкусу. 


Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

## Задание 7 [2 балла]. fastText

Используйте ```fastText``` в режиме классификации. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

## Задание 8 [4 балла]. CNN

Реализуйте модель Kim et al (2014) для решения задачи классификации с помощью CNN. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix).
Ссылка: Kim Y. Convolutional Neural Networks for Sentence Classification. 2014

## Задание 9 [4 + 2 балла]. RNN

(4 балла)Используйте ```RNN``` (BLSTM с какими-то признаками и пулинг поверх) для решения задачи текстовой классификации. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix).

За дополнительные 2 балла добавьте в модель символьные признаки - CharCNN или CharRNN

## Задание 10 [8 баллов]. ULMFit

Используйте ```ULMFit``` для решения задачи классификации. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

## Конец
Выполните какие-то из предыдущих заданий. Для всех заданий, кроме задания 1 требуется вычислить метрику accuracy метода.

Подведите итоги и проведите сравнение всех использованных методов. Какой из них показался вам лучше и почему?

**NB!** Задание обязательное вне зависимости от того, сколько из предыдущих пунктов вы выполнили, и дополнительных баллов не дает.


Для получения полной оценки за NLP-часть достаточно набрать **20 баллов**.

# Правила сдачи 

1. Домашняя работа должна быть выполнена в ipynb-тетрадке.
2. Сделанную тетрадку нужно отправить ассистенту (ссылка на контакты будет в вики).
3. Задание выполняется индивидуально.
4. Все вычисления должны быть снабжены пояснениями!
5. Дедлайн – 10 июня в 10.00.
